In [8]:
import numpy as np
import pandas as pd
import json
import csv
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
import pprint
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [9]:
# Reading the data
data = pd.read_csv("/content/drive/MyDrive/datasets/fake_news_detection_1.csv")

In [10]:
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [12]:
data = data.drop(["Unnamed: 0"], axis=1)

In [15]:
# encoding the labels
le = preprocessing.LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])

In [16]:
data.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1


In [21]:
data.shape

(6335, 3)

In [18]:
embedding_dim = 50
max_length = 54
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_size = len(data)
test_portion = .1

In [19]:
title = []
text = []
labels = []
for x in range(training_size):
	title.append(data['title'][x])
	text.append(data['text'][x])
	labels.append(data['label'][x])

In [22]:
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(title)
word_index1 = tokenizer1.word_index
vocab_size1 = len(word_index1)
sequences1 = tokenizer1.texts_to_sequences(title)
padded1 = pad_sequences(
	sequences1, padding=padding_type, truncating=trunc_type)
split = int(test_portion * training_size)
training_sequences1 = padded1[split:training_size]
test_sequences1 = padded1[0:split]
test_labels = labels[0:split]
training_labels = labels[split:training_size]

In [23]:
embeddings_index = {}
with open('/content/drive/MyDrive/datasets/glove.6B.50d.txt') as f:
	for line in f:
		values = line.split()
		word = values[0]
		coefs = np.asarray(values[1:], dtype='float32')
		embeddings_index[word] = coefs

# Generating embeddings
embeddings_matrix = np.zeros((vocab_size1+1, embedding_dim))
for word, i in word_index1.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embeddings_matrix[i] = embedding_vector

In [24]:
model = tf.keras.Sequential([
	tf.keras.layers.Embedding(vocab_size1+1, embedding_dim,
							input_length=max_length, weights=[
								embeddings_matrix],
							trainable=False),
	tf.keras.layers.Dropout(0.2),
	tf.keras.layers.Conv1D(64, 5, activation='relu'),
	tf.keras.layers.MaxPooling1D(pool_size=4),
	tf.keras.layers.LSTM(64),
	tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
			optimizer='adam', metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 54, 50)            586100    
                                                                 
 dropout (Dropout)           (None, 54, 50)            0         
                                                                 
 conv1d (Conv1D)             (None, 50, 64)            16064     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 64)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                        

In [43]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=5,
    patience=5,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

In [44]:
num_epochs = 50

training_padded = np.array(training_sequences1)
training_labels = np.array(training_labels)
testing_padded = np.array(test_sequences1)
testing_labels = np.array(test_labels)

history = model.fit(training_padded, training_labels,
					epochs=num_epochs,
					validation_data=(testing_padded, testing_labels), callbacks=[callback], verbose=2)

Train on 5702 samples, validate on 633 samples
Epoch 1/50
5702/5702 - 2s - loss: 0.0543 - accuracy: 0.9807 - val_loss: 0.7550 - val_accuracy: 0.7646 - 2s/epoch - 292us/sample
Epoch 2/50
5702/5702 - 2s - loss: 0.0458 - accuracy: 0.9846 - val_loss: 0.8659 - val_accuracy: 0.7599 - 2s/epoch - 295us/sample
Epoch 3/50
5702/5702 - 2s - loss: 0.0497 - accuracy: 0.9825 - val_loss: 0.8483 - val_accuracy: 0.7662 - 2s/epoch - 303us/sample
Epoch 4/50
5702/5702 - 3s - loss: 0.0580 - accuracy: 0.9774 - val_loss: 0.7896 - val_accuracy: 0.7662 - 3s/epoch - 443us/sample
Epoch 5/50
5702/5702 - 2s - loss: 0.0546 - accuracy: 0.9804 - val_loss: 0.8500 - val_accuracy: 0.7757 - 2s/epoch - 286us/sample
Epoch 6/50
5702/5702 - 2s - loss: 0.0542 - accuracy: 0.9805 - val_loss: 0.8899 - val_accuracy: 0.7725 - 2s/epoch - 290us/sample
Epoch 6: early stopping


In [48]:
# sample text to check if fake or not
X = "Saif and Rohan to buy tesla"

# detection
sequences = tokenizer1.texts_to_sequences([X])[0]
sequences = pad_sequences([sequences], maxlen=54,
						padding=padding_type,
						truncating=trunc_type)
if(model.predict(sequences, verbose=0)[0][0] >= 0.5):
	print("This news is True")
else:
	print("This news is false")

This news is false
